In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
#Read in tables
drivers = pd.read_csv('driver_ids.csv')
rides = pd.read_csv('ride_ids.csv')
rides_ts = pd.read_csv('ride_timestamps.csv')

In [3]:
#Get rid of rides with a negative distance
rides = rides[rides['ride_distance']>=0]
#Convert timestamps to datetime objects
drivers['driver_onboard_date'] = pd.to_datetime(drivers['driver_onboard_date'])
rides_ts['timestamp'] = pd.to_datetime(rides_ts['timestamp'])

In [4]:
#merge drivers and riders tables
drivers_and_rides = drivers.merge(rides, how='right', on='driver_id')
#calculate the estimated fare for each ride
meters_to_miles = 1609.344
seconds_to_mins = 60
base_fare = 2
mile_cost = 1.15
minute_cost = .22
service_fee = 1.75
drivers_and_rides['ride_distance'] = drivers_and_rides['ride_distance']/meters_to_miles
drivers_and_rides['ride_duration'] = drivers_and_rides['ride_duration']/seconds_to_mins
drivers_and_rides['ride_fare'] = drivers_and_rides['ride_distance']*mile_cost+\
    drivers_and_rides['ride_duration']*minute_cost+base_fare
drivers_and_rides['ride_fare']=drivers_and_rides['ride_fare']*\
    (1+drivers_and_rides['ride_prime_time']/100)+service_fee
drivers_and_rides['is_prime'] = drivers_and_rides['ride_prime_time']>0

In [5]:
#calculate the total prime time, total estimated fares and total number of rides
total_fares = drivers_and_rides.groupby('driver_id')['ride_fare'].apply(np.sum)
total_rides = drivers_and_rides.groupby('driver_id')['ride_fare'].apply(np.count_nonzero)
total_prime = drivers_and_rides.groupby('driver_id')['is_prime'].apply(np.mean)
total_fares = pd.DataFrame(total_fares)
total_fares = total_fares.rename(columns={'ride_fare': 'total_fares'})
total_rides = pd.DataFrame(total_rides)
total_rides = total_rides.rename(columns={'ride_fare': 'num_rides'})
total_prime = pd.DataFrame(total_prime)
total_prime = total_prime.rename(columns={'is_prime': 'percent_prime'})

In [6]:
#combine all the previous calculations into one table
driver_info = total_fares.merge(total_rides, how='inner', on='driver_id')
driver_info = driver_info.merge(total_prime, how='inner', on='driver_id')
driver_info['num_prime'] = np.round(driver_info['percent_prime']*driver_info['num_rides'])

In [7]:
#calculate average time between whem ride was accepted and when the rider was
#picked up for each driver
new_rides_ts = rides_ts.copy()
new_rides_ts = new_rides_ts.set_index('ride_id')
r_accepted = rides_ts[rides_ts['event']=='accepted_at']
r_picked = rides_ts[rides_ts['event']=='picked_up_at']
rides_pickup = r_accepted.merge(r_picked, how='inner', on='ride_id')
rides_pickup['pickup_time'] = rides_pickup['timestamp_y'] - rides_pickup['timestamp_x']
rides_pickup = rides_pickup[['ride_id', 'pickup_time']]
rides_pickup = rides_pickup.merge(rides, how='inner', on='ride_id')
driver_with_pickup = rides_pickup.merge(drivers, how='right', on='driver_id')
driver_with_pickup = pd.DataFrame(driver_with_pickup.groupby('driver_id')['pickup_time'].apply(np.mean))
#merge with rest of data
driver_full = driver_with_pickup.merge(driver_info, how='left', on='driver_id')
#convert pickup time to minutes
driver_full['pickup_time'] = driver_full['pickup_time'].dt.total_seconds().div(60)

In [8]:
driver_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 937 entries, 002be0ffdc997bd5c50703158b7c2491 to ffff51a71f2f185ec5e97d59dbcd7a78
Data columns (total 5 columns):
pickup_time      837 non-null float64
total_fares      854 non-null float64
num_rides        854 non-null float64
percent_prime    854 non-null float64
num_prime        854 non-null float64
dtypes: float64(5)
memory usage: 43.9+ KB


In [9]:
#calculate lifetime and ...
full_rides = rides.merge(rides_ts[rides_ts['event']=='dropped_off_at'], how='left', on='ride_id')
latest_rides = pd.DataFrame(full_rides.groupby('driver_id')['timestamp'].apply(np.max))
driver_lifetime = latest_rides.merge(drivers, how='right', on='driver_id')
driver_lifetime['lifetime']=driver_lifetime['timestamp']-driver_lifetime['driver_onboard_date']
end_time = np.max(driver_lifetime['timestamp'])
driver_lifetime['time_inactive']=end_time-driver_lifetime['timestamp']
driver_lifetime = driver_lifetime.set_index('driver_id')
driver_lifetime = driver_lifetime[['lifetime', 'time_inactive']]
#merge with rest of data
driver_full = driver_full.merge(driver_lifetime, how='left', on='driver_id')
#convert datetimes to days
driver_full['lifetime'] = driver_full['lifetime'].dt.days
driver_full['time_inactive'] = driver_full['time_inactive'].dt.days
#add rides per day column
driver_full['rides_per_day'] = driver_full['num_rides']/driver_full['lifetime']

In [10]:
driver_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 937 entries, 002be0ffdc997bd5c50703158b7c2491 to ffff51a71f2f185ec5e97d59dbcd7a78
Data columns (total 8 columns):
pickup_time      837 non-null float64
total_fares      854 non-null float64
num_rides        854 non-null float64
percent_prime    854 non-null float64
num_prime        854 non-null float64
lifetime         837 non-null float64
time_inactive    837 non-null float64
rides_per_day    837 non-null float64
dtypes: float64(8)
memory usage: 65.9+ KB


In [11]:
driver_full = driver_full.dropna(how='all')
driver_full[pd.isnull(driver_full['pickup_time'])]

,pickup_time,total_fares,num_rides,percent_prime,num_prime,lifetime,time_inactive,rides_per_day
driver_id,,,,,,,,
136b51093f684e15e2798e4dc1e23d0c,NaN,2120.082975,137.0,0.408759,56.0,NaN,NaN,NaN
1696be121baad60c7ca8a1c8164b24ad,NaN,690.742406,60.0,0.333333,20.0,NaN,NaN,NaN
1cf6fa07dcec364af2acf257b2d3731e,NaN,1195.096536,102.0,0.264706,27.0,NaN,NaN,NaN
23d3a2d0f6732d106fbc3d6079ac018d,NaN,912.290746,75.0,0.560000,42.0,NaN,NaN,NaN
2c00d6d77281fb9f97c1eb711f39b08d,NaN,934.652418,83.0,0.397590,33.0,NaN,NaN,NaN
4bbf15c7280e29c1df6edd7bf6dfa56a,NaN,759.133018,60.0,0.400000,24.0,NaN,NaN,NaN
4fc9091d4e900a41a207ee32a639d658,NaN,1247.967932,107.0,0.308411,33.0,NaN,NaN,NaN
53b03eb76e7c0e268c027a6868b9394c,NaN,1648.029748,147.0,0.326531,48.0,NaN,NaN,NaN
794a74f41f18a115252fd26bbd16882b,NaN,1857.397814,150.0,0.393333,59.0,NaN,NaN,NaN


In [12]:
#drop columns with null values
driver_full = driver_full.dropna(how='any')
driver_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 837 entries, 002be0ffdc997bd5c50703158b7c2491 to ffff51a71f2f185ec5e97d59dbcd7a78
Data columns (total 8 columns):
pickup_time      837 non-null float64
total_fares      837 non-null float64
num_rides        837 non-null float64
percent_prime    837 non-null float64
num_prime        837 non-null float64
lifetime         837 non-null float64
time_inactive    837 non-null float64
rides_per_day    837 non-null float64
dtypes: float64(8)
memory usage: 58.9+ KB


In [13]:
driver_full.to_csv('driver_full.csv')

In [14]:
driver_full.head()

,pickup_time,total_fares,num_rides,percent_prime,num_prime,lifetime,time_inactive,rides_per_day
driver_id,,,,,,,,
002be0ffdc997bd5c50703158b7c2491,2.970337,3560.546265,277.0,0.397112,110.0,86.0,3.0,3.220930
007f0389f9c7b03ef97098422f902e62,3.033871,321.494698,31.0,0.387097,12.0,85.0,4.0,0.364706
011e5c5dfc5c2c92501b8b24d47509bc,3.788725,482.427855,34.0,0.470588,16.0,68.0,14.0,0.500000
0152a2f305e71d26cc964f8d4411add9,4.000698,2608.898772,191.0,0.251309,48.0,64.0,0.0,2.984375
01674381af7edd264113d4e6ed55ecda,3.744933,5380.967089,375.0,0.264000,99.0,56.0,2.0,6.696429
